<a href="https://colab.research.google.com/github/Hesdi/KyrgyzNER/blob/main/dt_iii_bert_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATATHON-III: научи искусственный интеллект понимать кыргызский язык
### Пример решения на основе BERT

Сначала установим необходимые библиотеки, где многое сделано за нас

In [ ]:
! pip install -q deeppavlov==1.2.0 transformers==4.31.0 pytorch-crf==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.3/468.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Служебные функции:
- `get_sentences`: чтение из файла заданного (на хакатоне) формата
- `kyrgyz_ner_2_deeppavlov_ner`: преобразование в формат, с которым работает deeppavlov
- `write_predictions`: запись предсказаний в заданный файл в нужном формате, который примет система

In [ ]:
from itertools import groupby
from collections import namedtuple
from typing import List

# неизменяемый класс; в Sentence можно удобно "оборачивать" данные
Sentence = namedtuple("Sentence", "id tokens tags")


def get_sentences(filename: str) -> List[Sentence]:
    """
      filename - путь до файлов с данными
      Возвращает список предложений в структуре Sentence(id, tokens, tags).
    """
    with open(filename, "r", encoding="utf-8") as inp:
        lines = inp.readlines()

    sentences = [list(sentence) for k, sentence in groupby(lines, key=lambda x: x.strip() != "") if k]
    sentences = [Sentence(s[0].strip(),
                          [t.split("\t")[0].strip() for t in s[1:]],
                          [t.split("\t")[-1].strip() for t in s[1:]]) for s in sentences]

    return sentences


def kyrgyz_ner_2_deeppavlov_ner(source: str, destination: str) -> None:
    """
      source - путь до train-файла, который нужно сконвертировать в формат,
      пригодный для deeppavlov NER, чтобы можно было обучаться

      destination - путь, куда нужно сохранить сконвертированный файл

      Преобразовывает исходные данные в формат, который понимает система
      deeppavlov NER.
    """
    sentences = get_sentences(source)

    with open(destination, "w", encoding="utf-8") as out:
        for s in sentences:
            for tok, tag in zip(s.tokens, s.tags):
                print(tok, tag, file=out)
            print("", file=out)

def write_predictions(sentences: List[Sentence], predictions, destination: str):
    """
      sentences - список предложений, полученный из `get_sentences`
      predictions - предсказания, полученные из обученной NER модели
      destination - путь, по которому будет записан файл с предсказаниями
    """
    print(type(predictions), predictions)
    with open(destination, "w", encoding="utf-8") as out:
        for sent, pred in zip(sentences, predictions):
            print(sent.id, file=out)
            for tok, p in zip(sent.tokens, pred):
                print(f"{tok}\t-\t-\t{p}", file=out)
            print("", file=out)


In [ ]:
import os

# Создаем директорию, куда положим данные в исправленном формате
data_dir = "train_data"
os.makedirs(data_dir, exist_ok=True)

# Здесь следует указать пути до файлов
kyrgyz_ner_2_deeppavlov_ner("gold.conll2003-formatted.30-texts.txt", f"{data_dir}/train.txt")
kyrgyz_ner_2_deeppavlov_ner("gold.conll2003-formatted.30-texts.txt", f"{data_dir}/valid.txt")
kyrgyz_ner_2_deeppavlov_ner("gold.conll2003-formatted.30-texts.txt", f"{data_dir}/test.txt")

Подготовив всё необходимое, возьмём готовую модель (из библиотеки DeepPavlov), которую можно дообучать.

[Статья с описанием](http://www.ijmlc.org/vol9/758-ML0025.pdf) подхода для LSTM, а для BERT сделано по аналогии.

Вот [ссылка на конфигурацию](
https://github.com/deeppavlov/DeepPavlov/blob/master/deeppavlov/configs/ner/ner_ontonotes_bert_mult.json), может, можно поменять что-то ещё?

In [ ]:
from deeppavlov.core.commands.utils import parse_config

# Загружаем конфигурацию и заменяем путь до обучающих данных на свой
model_config = parse_config("ner_ontonotes_bert_mult")
model_config["dataset_reader"]["data_path"] = data_dir
model_config["train"]["epochs"] = 5
model_config["train"]["batch_size"] = 16

model_config["train"]

{'epochs': 5,
 'batch_size': 16,
 'metrics': [{'name': 'ner_f1', 'inputs': ['y', 'y_pred']},
  {'name': 'ner_token_f1', 'inputs': ['y', 'y_pred']}],
 'validation_patience': 100,
 'val_every_n_batches': 20,
 'log_every_n_batches': 20,
 'show_examples': False,
 'pytest_max_batches': 2,
 'pytest_batch_size': 8,
 'evaluation_targets': ['valid', 'test'],
 'class_name': 'torch_trainer'}

### Обучение модели

In [ ]:
from deeppavlov import train_model

ner_model = train_model(model_config)

2023-07-26 09:54:48.511 WARNING in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 66: TorchTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


2023-07-26 09:55:04.54 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 104: [saving vocabulary to /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[saving vocabulary to /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf/tag.dict]


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-07-26 09:55:17.661 WARNING in 'deeppavlov.models.torch_bert.torch_transformers_sequence_tagger'['torch_transformers_sequence_tagger'] at line 310: Init from scratch. Load path /root/.deeppavlov/models/ner_ontonotes_torch_bert_mult_crf/model_crf.pth.tar does not exist.
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)
14it [00:04,  3.07it/s]
2023-07-26 09:

{"valid": {"eval_examples_count": 209, "metrics": {"ner_f1": 0.7836, "ner_token_f1": 4.733}, "time_spent": "0:00:05", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 100}}


INFO:train_report:{"valid": {"eval_examples_count": 209, "metrics": {"ner_f1": 0.7836, "ner_token_f1": 4.733}, "time_spent": "0:00:05", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 100}}
14it [00:03,  3.61it/s]
5it [00:01,  3.89it/s]
1it [00:00, 11.65it/s]

...

INFO:train_report:{"valid": {"eval_examples_count": 209, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 80.3725}, "time_spent": "0:00:53", "epochs_done": 4, "batches_seen": 60, "train_examples_seen": 900, "impatience": 0, "patience_limit": 100}}
14it [00:19,  1.37s/it]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
14it [00:01, 11.34it/s]


{"valid": {"eval_examples_count": 209, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 80.3725}, "time_spent": "0:00:02"}}


INFO:train_report:{"valid": {"eval_examples_count": 209, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 80.3725}, "time_spent": "0:00:02"}}
14it [00:01, 11.83it/s]


{"test": {"eval_examples_count": 209, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 80.3725}, "time_spent": "0:00:02"}}


INFO:train_report:{"test": {"eval_examples_count": 209, "metrics": {"ner_f1": 72.7273, "ner_token_f1": 80.3725}, "time_spent": "0:00:02"}}
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Предсказания для тестовой выборки

In [ ]:
sentences = get_sentences("gold.conll2003-formatted.txt")
_, predictions = ner_model([s.tokens for s in sentences])

Записываем предсказанные результаты в выходной файле в формате хакатона

In [ ]:
write_predictions(sentences, predictions, "predictions.txt")

<class 'list'> [['I-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'B-TITLE', 'O', 'O', 'O'], ['O', 'B-INSTITUTION', 'I-INSTITUTION', 'I-INSTITUTION', 'I-INSTITUTION', 'I-INSTITUTION', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PERSON', 'I-PERSON', 'B-LOCATION', 'B-INSTITUTION', 'I-INSTITUTION', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'O', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'O', 'O', 'B-INSTITUTION', 'I-INSTITUTION', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERSON', 'O', 'B-PERSON', 'I-PERS